# Creating a dataset with a simple format 
In this tutorial, we will walk through how to generate a dataset using a simple input format that will work with some parts of the FEMR pipeline.

In [1]:
import os
import csv

# Create directories to save the dataset and associated files 
TARGET_DIR = "trash/simple_femr"
os.makedirs(TARGET_DIR)

INPUT_DIR = os.path.join(TARGET_DIR, "simple_input")
os.mkdir(INPUT_DIR)

# 1. Basic input schema
The input schema is a folder of csv files, where each csv file has <u>at minimum</u> the following columns:

>patient_id, start, code, value

where 
 - `patient_id` is the ID for the patient who has the event. `patient_id` must be a 64 bit unsigned integer

 - `start` is the start timestamp for an event, ideally when the event is initially recorded in the database. `start` must be an ISO 8601 timestamp string

 - `code` is a string identifies what the event is. It must internally consist of a vocabulary signifier and the code itself, split by a "/" character. For example ICD10CM/E11.4

 - `value` is a value associated with the event. It can either be a numeric value, an arbitrary string, or an empty string (indicating no value).

Each row in a file corresponds to an event.


All different types of EMR data can be mapped to four core columns: 

>Demographics, Diagnosis/Procedures, Labs, Medications

and we will demonstrate how to add each data type to a dataset in a sequence. 



# Exercise 1: Add demographic information
Now, we create a dataset, "minimum.csv", manually that contains a single patient with three rows. 

We use patients' date of birth as the very first visit time in our data format, so the first event is always `Birth`

Rows 2 and 3 include the demographic information, `Gender` and `Race`, respectively, of this patient with corresponding values. Demographics should generally be mapped as demographics codes assigned to the birth date of the patient (with no value assigned). For demographic rows, the vocabulary signifier and the code itself are the same.

In [4]:
with open(os.path.join(INPUT_DIR, "minimum.csv"), "w") as f:
    f.write("patient_id,start,code,value\n")              
    f.write("3,1990-01-07,Birth/Birth,\n")         # First event is always birth
    f.write("3,1990-01-07,Gender/Gender,Female\n") # demographics 
    f.write("3,1990-01-07,Race/Race,White\n")      # demographics
    
with open(os.path.join(INPUT_DIR, "minimum.csv"), newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

patient_id,start,code,value
3,1990-01-07,Birth/Birth,
3,1990-01-07,Gender/Gender,Female
3,1990-01-07,Race/Race,White


# 2. Expanding your dataset
You may also add arbitrary columns for any csv file. The additional columns will be added to each event. The columns can vary between csv files. 


# Exercise 2: Add diagnosis information
We now add more events/rows that capture patients' diagnosis information, e.g., ICD 9/10 codes

For diagnosis, the `code` column has two parts, the vocabulary signifier (e.g., ICD10CM) and the code itself (e.g., E11.4, E10.1, etc.) The `value` column should be left as empty. Procedures and diagnosis codes should generally be mapped to when the event happened.

Note that different diagnoses may be given at different visits, so the corresponding `start` timestamps may be different.

In [5]:
with open(os.path.join(INPUT_DIR, "diagnosis.csv"), "w") as f:
    f.write("patient_id,start,code,value\n")              
    f.write("3,1970-01-07,Birth/Birth,\n")         
    f.write("3,1990-01-07,Gender/Gender,Female\n") 
    f.write("3,1990-01-07,Race/Race,White\n")      
    f.write("3,2022-05-03,ICD10CM/E11.4,\n")  # diabetes
    f.write("3,2022-06-05,ICD10CM/E10.1,\n")  # hypertension 
    
with open(os.path.join(INPUT_DIR, "diagnosis.csv"), newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

patient_id,start,code,value
3,1970-01-07,Birth/Birth,
3,1990-01-07,Gender/Gender,Female
3,1990-01-07,Race/Race,White
3,2022-05-03,ICD10CM/E11.4,
3,2022-06-05,ICD10CM/E10.1,


# Exercise 3: Add lab test information
We now add more events/rows that capture patients' lab values, e.g., Vitals/Blood Pressure

For vitals, the `code` column has two parts, the vocabulary signifier (e.g., Vitals) and the code itself (e.g., Blood Pressure, HbA1c, etc.) The `value` column should contain corresponding numeric values when possible, but text valued otherwise.

We also recommend adding another column, `units`, to record the units of each test result.

Note that different lab tests may be given at different visits, so the corresponding `start` timestamps may differ.

In [6]:
with open(os.path.join(INPUT_DIR, "lab_tests.csv"), "w") as f:
    f.write("patient_id,start,code,value,units\n")              
    f.write("3,1970-01-07,Birth/Birth,,\n")         
    f.write("3,1990-01-07,Gender/Gender,Female,\n") 
    f.write("3,1990-01-07,Race/Race,White,\n")      
    f.write("3,2022-05-03,ICD10CM/E11.4,,\n")  
    f.write("3,2022-06-05,ICD10CM/E10.1,,\n") 
    f.write("3,2000-07-09,Vitals/Blood Pressure,160,mmHg\n") 
    f.write("3,2000-08-09,Vitals/HbA1c,7,%\n") 
    
with open(os.path.join(INPUT_DIR, "lab_tests.csv"), newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

patient_id,start,code,value,units
3,1970-01-07,Birth/Birth,,
3,1990-01-07,Gender/Gender,Female,
3,1990-01-07,Race/Race,White,
3,2022-05-03,ICD10CM/E11.4,,
3,2022-06-05,ICD10CM/E10.1,,
3,2000-07-09,Vitals/Blood, Pressure,160,mmHg
3,2000-08-09,Vitals/HbA1c,7,%


# Exercise 4: Add medication information
We now add more events/rows that capture patients' medication intake info., e.g., Drug/Atorvastatin

For medications, the `code` column has two parts, the vocabulary signifier (e.g., Drug) and the code itself (e.g., Atorvastatin, Heparin Lock Flush, Multivitamins, etc.) The `value` column should be empty.

We add another column, `dosage`, to record the dose of the prescribed medication. For medications, the `units` column indicates the unit of the medication dose.

Note that different lab tests may be given at different visits, so the corresponding `start` timestamps may differ.

In [7]:
with open(os.path.join(INPUT_DIR, "drug.csv"), "w") as f:
    f.write("patient_id,start,code,value,units,dosage\n")              
    f.write("3,1970-01-07,Birth/Birth,,,\n")         
    f.write("3,1990-01-07,Gender/Gender,Female,,\n") 
    f.write("3,1990-01-07,Race/Race,White,,\n")      
    f.write("3,2022-05-03,ICD10CM/E11.4,,,\n")  
    f.write("3,2022-06-05,ICD10CM/E10.1,,,\n") 
    f.write("3,2020-07-09,Vitals/Blood Pressure,160,mmHg,\n") 
    f.write("3,2020-08-09,Vitals/HbA1c,7,,%\n") 
    f.write("3,2022-06-05,Drug/Atorvastatin,,mg,50\n") 
    f.write("3,2022-07-06,Drug/Multivitamins,,ml,5\n")
    
with open(os.path.join(INPUT_DIR, "drug.csv"), newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

patient_id,start,code,value,units,dosage
3,1970-01-07,Birth/Birth,,,
3,1990-01-07,Gender/Gender,Female,,
3,1990-01-07,Race/Race,White,,
3,2022-05-03,ICD10CM/E11.4,,,
3,2022-06-05,ICD10CM/E10.1,,,
3,2020-07-09,Vitals/Blood, Pressure,160,mmHg,
3,2020-08-09,Vitals/HbA1c,7,,%
3,2022-06-05,Drug/Atorvastatin,,mg,50
3,2022-07-06,Drug/Multivitamins,,ml,5


# 3. Additional considerations
When adding columns, the goal is to add as many columns as needed to maximum the clarity and completeness of the raw/original data. 

For simplicity, we only included one patient in the above datasets, but an arbitrary number of patients can be captured in the same file by adding more rows with different `patient_id`.


# Exercise 5: Generate synthetic examples
We now create a dataset with 100 synthetic patients' data

In [2]:
import random
import datetime
import string

def add_random_patient(patient_id, f):
    epoch = datetime.date(1990, 1, 1)
    birth = epoch + datetime.timedelta(days=random.randint(100, 1000))
    current_date = birth
    
    code_cat = ["Birth","Gender", "Race","ICD10CM","CPT", "Drug", "Vitals"]
    gender_values = ["F","M"]
    race_values = ["White","Non-White"]
    index = random.randint(0,1)
    for code_type in code_cat:
        if code_type == "Birth":
            clarity = "PATIENT"
            visit_id = 1
            value = ''
            dosage = ''
            unit = ''
            f.write(f"{patient_id},{birth.isoformat()},{code_type}/{code_type},{value},{dosage},{visit_id},{unit},{clarity}\n")
        elif code_type == "Gender":
            clarity = "PATIENT"
            visit_id = 1
            value = gender_values[index]
            dosage = ''
            unit = ''
            f.write(f"{patient_id},{birth.isoformat()},{code_type}/{code_type},{value},{dosage},{visit_id},{unit},{clarity}\n")
        elif code_type == "Race":
            clarity = "PATIENT"
            visit_id = 1
            value = race_values[index]
            dosage = ''
            unit = ''
            f.write(f"{patient_id},{birth.isoformat()},{code_type}/{code_type},{value},{dosage},{visit_id},{unit},{clarity}\n") 
        else:
            for code in range(random.randint(1, 10)):
                code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
                current_date = current_date +  datetime.timedelta(days=random.randint(0, 100))
                visit_id = random.randint(0,20)
                if code_type == "ICD10CM":
                    clarity = "DIAGNOSIS"
                    value = ''
                    dosage = ''
                    unit = ''
                elif code_type == "CPT":
                    clarity = "PROCEDURES"
                    value = ''
                    dosage = ''
                    unit = ''
                elif code_type == "Drug":
                    clarity = "MED_ORDER"
                    value = ''
                    dosage = random.randint(10,50)
                    unit = "mg"
                elif code_type == "Vitals":
                    clarity = "LAB_RESULT"
                    value = random.randint(80,200)
                    dosage = ''
                    unit = "mmHg"
                f.write(f"{patient_id},{current_date.isoformat()},{code_type}/{code},{value},{dosage},{visit_id},{unit},{clarity}\n") 


In [5]:
for file_no in range(1, 2):
    with open(os.path.join(INPUT_DIR, f"{file_no}.csv"), "w") as f:    
        # Use the simpler 4 column minimum format, with no optional columns
        f.write("patient_id,start,code,value,dosage,visit_ids,lab_units,clarity_source\n")
        for patient_id in range(file_no * 100, (file_no + 1) * 100):
            add_random_patient(patient_id, f)
        
        with open(os.path.join(INPUT_DIR, f"{file_no}.csv"), newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            for row in spamreader:
                print(', '.join(row))

patient_id,start,code,value,dosage,visit_ids,lab_units,clarity_source
100,1992-08-28,Birth/Birth,,,1,,PATIENT
100,1992-08-28,Gender/Gender,F,,1,,PATIENT
100,1992-08-28,Race/Race,White,,1,,PATIENT
100,1992-11-15,ICD10CM/8RGSN32K,,,13,,DIAGNOSIS
100,1993-01-30,ICD10CM/K59YO0LH,,,4,,DIAGNOSIS
100,1993-03-20,CPT/CVGA0L2C,,,2,,PROCEDURES
100,1993-03-27,CPT/NW9ITAQ3,,,7,,PROCEDURES
100,1993-06-18,CPT/L85SUOU2,,,3,,PROCEDURES
100,1993-08-31,CPT/7I2C7X33,,,20,,PROCEDURES
100,1993-08-31,CPT/BJAGWXD4,,,15,,PROCEDURES
100,1993-09-24,Drug/LAC2GO6U,,22,14,mg,MED_ORDER
100,1993-10-18,Drug/U6M468D7,,13,7,mg,MED_ORDER
100,1993-12-09,Drug/5RI7VTP5,,41,15,mg,MED_ORDER
100,1994-02-21,Drug/29FN1ZI0,,15,19,mg,MED_ORDER
100,1994-03-24,Drug/OKCY9XAF,,24,0,mg,MED_ORDER
100,1994-05-10,Drug/V619Q4IJ,,14,9,mg,MED_ORDER
100,1994-07-08,Drug/2V1CAW36,,23,9,mg,MED_ORDER
100,1994-08-04,Drug/6DKT18ZI,,44,16,mg,MED_ORDER
100,1994-10-14,Drug/5UIC6BEW,,29,9,mg,MED_ORDER
100,1994-12-19,Drug/O4ATG34K,,10,10,mg,MED_ORDER
10

# Additional notes
Ordering of rows for each patient, or patient rows being split across files doesn't matter. Everything will be resorted and joined as part of the ETL process.

Other atypical datatypes, such as flowsheets can be added as needed, with either string or numeric values as whatever is more natural.

# 4. Convert the directory to an extract
We now convert the dataset we created above to an extract using the function [etl_simple_femr](https://github.com/som-shahlab/femr/blob/main/src/femr/etl_pipelines/simple.py#L66) from the femr repo

The output extract is a femr [PatientDatabase](https://github.com/som-shahlab/femr/blob/Miking98-patch-1/tutorials/0_How%20FEMR%20Works%20%2B%20Toy%20Example.ipynb) that can be directly used by the femr pipeline

In [6]:
# Create directories for storing the extract and extract log
LOG_DIR = os.path.join(TARGET_DIR, "logs")
EXTRACT_DIR = os.path.join(TARGET_DIR, "extract")

import femr
import femr.etl_pipelines.simple
os.system(f"etl_simple_femr {INPUT_DIR} {EXTRACT_DIR} {LOG_DIR} --num_threads 2")

2023-04-19 16:23:04,449 [MainThread  ] [INFO ]  Extracting from OMOP with arguments Namespace(simple_source='trash/simple_femr/simple_input', target_location='/local-scratch/nigam/projects/yizhex/trash/simple_femr/extract', temp_location='/local-scratch/nigam/projects/yizhex/trash/simple_femr/logs', num_threads=2)
2023-04-19 16:23:04,450 [MainThread  ] [INFO ]  Converting to events
2023-04-19 16:23:04,536 [MainThread  ] [INFO ]  Converting to patients
2023-04-19 16:23:04,575 [MainThread  ] [INFO ]  Converting to extract


Done with main 2023-04-19T16:23:04.623173171+00:00
Done with meta 2023-04-19T16:23:04.623279935+00:00
Converting to extract 2023-04-19 16:23:04.575945


0

# 5. Open and view the data
We now open and take a look at the femr extract we generated in the last step using the function [PatientDatabase](https://github.com/som-shahlab/femr/blob/main/src/femr/extension/datasets.pyi#L24) from the femr repo

In [7]:
import femr.datasets

database = femr.datasets.PatientDatabase(EXTRACT_DIR)

# Number of patients
print("Num patients", len(database))

# Print out the first patient
patient = database[0]
print(patient)

# Note that the patient ids get remapped, you can unmap with the database
original_id = database.get_original_patient_id(0)
print("Oiringla id for patient 0", original_id)

# Also note that concepts have been mapped to integers
print("What code 3 means", database.get_code_dictionary()[3])  # Returns what code=3 means in the database

# You can pull things like dosage by looking at the event
for event in patient.events:
    print(event, event.dosage)


Num patients 100
Patient(patient_id=0, events=(Event(start=1990-11-04 00:00:00, code=1, value=Non-White, dosage=, visit_ids=1, lab_units=, clarity_source=PATIENT), Event(start=1990-11-04 00:00:00, code=0, dosage=, visit_ids=1, lab_units=, clarity_source=PATIENT), Event(start=1990-11-04 00:00:00, code=2, value=M, dosage=, visit_ids=1, lab_units=, clarity_source=PATIENT), Event(start=1990-12-26 00:00:00, code=202, dosage=, visit_ids=20, lab_units=, clarity_source=DIAGNOSIS), Event(start=1990-12-31 00:00:00, code=1130, dosage=, visit_ids=17, lab_units=, clarity_source=DIAGNOSIS), Event(start=1991-01-31 00:00:00, code=1643, dosage=, visit_ids=1, lab_units=, clarity_source=DIAGNOSIS), Event(start=1991-04-22 00:00:00, code=728, dosage=, visit_ids=13, lab_units=, clarity_source=DIAGNOSIS), Event(start=1991-05-11 00:00:00, code=710, dosage=, visit_ids=11, lab_units=, clarity_source=DIAGNOSIS), Event(start=1991-06-30 00:00:00, code=20, dosage=, visit_ids=20, lab_units=, clarity_source=DIAGNOSIS